In [1]:
import pImpactR as impact
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy as copy
import shutil 
import pickle
rectangular_ = 1
elliptic_ = 2

In [2]:
NL_t = 0.4
emitGeomRMS = 3.3e-6
npt = 1000000
print('npt=',npt)
nturn = 10000

npt= 1000000


In [3]:
NL_L = 1.8
NL_c = 0.01
NL_nu = 0.3

# read Aperture

In [4]:
aperture = np.loadtxt('pipeinfo.in')

In [5]:
aperture[-1,0]

39.968229720000004

# read impact lattice

In [6]:
beam,lattice = impact.readInputFile('test.Chad.in')
beam.nCore_y=8
beam.nCore_z=4
beam.n_particles = npt

ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

g = ke/mass+1.0
bg = np.sqrt(g**2-1.0)
emitN = emitGeomRMS*bg
print('\n\n emitN =',emitN)

reading ImpactZ input file (test.Chad.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


 emitN = 2.4105916749973357e-07


In [7]:
cleanLat = impact.clearLattice(lattice)
L = 0 
for item in cleanLat:
    if item.type == 'RFkick':
        item.vmax = 0.0
    if item.type == 'nonlinear_insert':
        NLfoward = item
        print('NL:',L,L+item.length)
    if 'length' in item:
        L = L+item.length
print(L-aperture[-1,0])

NLfoward.n_sckick = 2
NLfoward.n_map = 45

NL: 4.9115191429 6.711519142899999
-4.199918635094946e-09


In [8]:
NLfoward


             length: 1.8 [m]
           n_sckick: 2 [1]
              n_map: 45 [1]
         strength_t: 0.4
 transverse_scale_c: 0.01
        pipe_radius: 10.0 [m]
       tune_advance: 0.3
               type: 'nonlinear_insert'

In [9]:
for item in cleanLat:
    if 'length' in item:
        item.n_sckick = int(np.ceil(item.length*50))
        item.n_map = 1

add quadrupole fringe hard-edge model

In [10]:
cleanLat = impact.addHardEdgeQuad(cleanLat)

# Back tracking for IOTA matched beam at injection location

In [11]:
for i in range(len(cleanLat)):
    if cleanLat[i].type == 'nonlinear_insert':
        break

latticeB = impact.getInverseLattice(cleanLat[:i])

write0 = impact.getElem('write_raw_ptcl')
write0.file_id = 111110
write0.format_id = 2
latticeB.insert(0,write0)


write1 = impact.getElem('write_raw_ptcl')
write1.file_id = 111111
write1.format_id = 2
latticeB.append(write1)

loop = impact.getElem('loop')
loop.turns = 1
latticeB.insert(0,loop)

In [12]:
cleanLat.insert(0,impact.getElem('pipeinfo'))
cleanLat.insert(1,impact.getElem('loop'))
cleanLat[1].turns = nturn

write2 = impact.getElem('write_raw_ptcl')
write2.file_id = 222222
write2.format_id = 2
cleanLat.append(write2)

# Define runs

### thermal

In [13]:
def prepareIOTA_Thermal(tauBeam,tauMag,H0,dE,order,CL):
    beam.distribution.mode = 'twiss'
    beam.distribution.distribution_type = 'IOTA_Gauss'
    beam.distribution.NL_t  = tauBeam
    beam.distribution.NL_c  = NL_c
    beam.distribution.betx  = NL_L/np.sin(2.0*np.pi*NL_nu)
    beam.distribution.alfx  = np.tan(np.pi*NL_nu)
    beam.distribution.emitx = H0
    beam.distribution.CL    = CL
    beam.distribution.betz  = 200
    beam.distribution.alfz  = 0.0
    beam.distribution.emitz = (dE*ke*1.0e-6)**2*beam.distribution.betz
    impact.writeInputFile(beam,latticeB)
    impact.writeInputFile(beam,latticeB,'test.in.prepare.nonlinearThermal.tau='+str(tauBeam))
    impact.run(beam,order=order)
    pData0 = impact.readParticleData(111110,ke,mass,freq,2)
    Twiss0 = impact.getTwiss_from_pData(pData0)
    pData0 = pData0[:,[0,1,2,3,5,-1]]
    pData1 = impact.readParticleData(111111,ke,mass,freq,2)
    Twiss1 = impact.getTwiss_from_pData(pData1)
    pData1 = pData1[:,[0,1,2,3,5,-1]]
    
    beam.distribution.distribution_type = 'ReadFile_binary'
    beam.distribution.file_id = 111111
    NLfoward.strength_t = tauMag
    impact.writeInputFile(beam,cleanLat,'test.in.run.tau='+str(tauBeam))
    
    shutil.copyfile('fort.111110', 'fort.NL_ent.thermal.npt='+str(npt)+'.NL_t='+str(tauMag)+'.beam_t='+str(tauBeam)+'.turn=0')
    shutil.copyfile('fort.111111', 'fort.inject.thermal.npt='+str(npt)+'.NL_t='+str(tauMag)+'.beam_t='+str(tauBeam)+'.turn=0')
    
    return 0

In [14]:
def runIOTA_Thermal(t0,t1,tauBeam,tauMag,order):
    
    shutil.copyfile('fort.inject.thermal'+
                    '.npt='+str(npt)+
                    '.NL_t='+str(tauMag)+
                    '.beam_t='+str(tauBeam)+
                    '.turn='+str(t0),
                    'fort.111111')
    
    beam.distribution.distribution_type = 'ReadFile_binary'
    beam.distribution.file_id = 111111
    NLfoward.strength_t = tauMag
    cleanLat[1].turns = t1-t0
    write2.turn = t1-t0
    impact.writeInputFile(beam,cleanLat)
    impact.run(beam,order=order)
    
    try:
        pLost = np.loadtxt('lost_partcl.data',skiprows=1, usecols=(0,1,2,3))
        print(len(pLost))
    except:
        pLost = 0
        
    fname = 'lost_partcl.pickle.thermal'+   \
            '.NL_t='+str(tauMag)+    \
            '.beam_t='+str(tauBeam)+ \
            '.t0='+str(t0)+          \
            '.t1='+str(t1)
    pickle.dump(pLost,open(fname,'wb'))
    
    
    shutil.copyfile('fort.222222', 
                    'fort.inject.thermal'+
                    '.npt='+str(npt)+
                    '.NL_t='+str(tauMag)+
                    '.beam_t='+str(tauBeam)+
                    '.turn='+str(t1))
    
    return 0

In [23]:
def getIOTA_Thermal_data(T,tauBeam,tauMag):
    '''
    T=[t0,t1,t2...tn]
    '''
    shutil.copyfile('fort.NL_ent.thermal.npt='+str(npt)+'.NL_t='+str(tauMag)+'.beam_t='+str(tauBeam)+'.turn=0',
                    'fort.111110')
    shutil.copyfile('fort.inject.thermal.npt='+str(npt)+'.NL_t='+str(tauMag)+'.beam_t='+str(tauBeam)+'.turn=0',
                    'fort.111111')
    
    pData0 = impact.readParticleData(111110,ke,mass,freq,2)
    Twiss0 = impact.getTwiss_from_pData(pData0)
    pData0 = pData0[:,[0,1,2,3,5,-1]]
    pData1 = impact.readParticleData(111111,ke,mass,freq,2)
    Twiss1 = impact.getTwiss_from_pData(pData1)
    pData1 = pData1[:,[0,1,2,3,5,-1]]
    
    pLostList = []
    nptLost = []
    n = 0
    for i in range(len(T)-1):
        t0 = T[i]
        t1 = T[i+1]
        fname = 'lost_partcl.pickle.thermal'+   \
                '.NL_t='+str(tauMag)+    \
                '.beam_t='+str(tauBeam)+ \
                '.t0='+str(t0)+          \
                '.t1='+str(t1)
        tmp = pickle.load(open(fname,'rb'))
        if not isinstance(tmp,int):
            nptLost.append(len(tmp))
            pLostList.append(tmp)
        if len(nptLost)>1:
            nptLost[-1]=nptLost[-2]+nptLost[-1]
    
    pLost = np.zeros([nptLost[-1],4])
    pLost[:nptLost[0],:]=pLostList[0]
    for i in range(1,len(pLostList)):
        pLost[nptLost[i-1]:nptLost[i],:]=pLostList[i]
    
        
    return {'Twiss0':Twiss0,'Twiss1':Twiss1,'pData0':pData0,'pData1':pData1,'pLost':pLost}

### Exponential

In [16]:
def prepare_Exponential(betx,alfx,emitx,CLx,bety,alfy,emity,CLy,dE,tauBeam,tauMag,order):
    beam.distribution.mode = 'twiss'
    beam.distribution.distribution_type = 'Exponential2D_trunc'
    beam.distribution.betx  = betx
    beam.distribution.alfx  = alfx
    beam.distribution.emitx = emitx
    beam.distribution.CLx   = CLx
    beam.distribution.bety  = bety
    beam.distribution.alfy  = alfy
    beam.distribution.emity = emity
    beam.distribution.CLy   = CLy
    beam.distribution.betz  = 200
    beam.distribution.alfz  = 0.0
    beam.distribution.emitz = (dE*ke*1.0e-6)**2*beam.distribution.betz
    
    impact.writeInputFile(beam,cleanLat[:2]+[write1])
    impact.run(beam,order=order)

    shutil.copyfile('fort.111111', 'fort.inject.exponential'+
                                   '.npt='+str(npt)+
                                   '.NL_t='+str(tauMag)+
                                   '.beam_t='+str(tauBeam)+'.turn0')

    return 0

In [17]:
def run_Exponential(t0,t1,tauBeam,tauMag,order):
    
    shutil.copyfile('fort.inject.exponential'+
                    '.npt='+str(npt)+
                    '.NL_t='+str(tauMag)+
                    '.beam_t='+str(tauBeam)+
                    '.turn='+str(t0),
                    'fort.111111')
    
    beam.distribution.distribution_type = 'ReadFile_binary'
    beam.distribution.file_id = 111111
    NLfoward.strength_t = tauMag
    cleanLat[1].turns = t1-t0
    write2.turn = t1-t0
    impact.writeInputFile(beam,cleanLat)
    impact.run(beam,order=order)
    
    try:
        pLost = np.loadtxt('lost_partcl.data',skiprows=1, usecols=(0,1,2,3))
        print(len(pLost))
    except:
        pLost = 0
        
    fname = 'lost_partcl.pickle.exponential'+   \
            '.NL_t='+str(tauMag)+    \
            '.beam_t='+str(tauBeam)+ \
            '.t0='+str(t0)+          \
            '.t1='+str(t1)
    pickle.dump(pLost,open(fname,'wb'))
    
    
    shutil.copyfile('fort.222222', 
                    'fort.inject.exponential'+
                    '.npt='+str(npt)+
                    '.NL_t='+str(tauMag)+
                    '.beam_t='+str(tauBeam)+
                    '.turn='+str(t1))
    
    return 0

In [18]:
def get_Exponential_data(T,tauBeam,tauMag):
    '''
    T=[t0,t1,t2...tn]
    '''
    shutil.copyfile('fort.inject.exponential.npt='+str(npt)+'.NL_t='+str(tauMag)+'.beam_t='+str(tauBeam)+'.turn=0',
                    'fort.111111')
    
    pData1 = impact.readParticleData(111111,ke,mass,freq,2)
    Twiss1 = impact.getTwiss_from_pData(pData1)
    pData1 = pData1[:,[0,1,2,3,5,-1]]
    
    pLostList = []
    nptLost = []
    n = 0
    for i in range(len(T)-1):
        t0 = T[i]
        t1 = T[i+1]
        fname = 'lost_partcl.pickle.exponential'+   \
                '.NL_t='+str(tauMag)+    \
                '.beam_t='+str(tauBeam)+ \
                '.t0='+str(t0)+          \
                '.t1='+str(t1)
        tmp = pickle.load(open(fname,'rb'))
        if not isinstance(tmp,int):
            nptLost.append(len(tmp))
            pLostList.append(tmp)
        if len(nptLost)>1:
            nptLost[-1]=nptLost[-2]+nptLost[-1]
    
    pLost = np.zeros([nptLost[-1],4])
    pLost[:nptLost[0],:]=pLostList[0]
    for i in range(1,len(pLostList)):
        pLost[nptLost[i-1]:nptLost[i],:]=pLostList[i]
    
        
    return {'Twiss1':Twiss1,'pData1':pData1,'pLost':pLost}

# Run and collect data

In [19]:
# lostp_data = {}
import pickle
# pickle.dump(lostp_data,open('lostp_data.QFF.p100000.pickle','wb'))
# lostp_data = pickle.load(open('lostp_data.QFF.p100000.pickle','rb'))

### Thermal, $H_{\tau=0.4}$, $\sigma_E=2\times10^{-3}$, $order=3$

In [20]:
prepareIOTA_Thermal(0.4,0.4,emitGeomRMS,2.0e-3,3,6.0)

0

In [ ]:
runIOTA_Thermal(0,1000,0.4,0.4,3)

In [24]:
data = getIOTA_Thermal_data([0,1,2,3],0.4,0.4)

In [26]:
Twiss1=data['Twiss1']
Twiss1

(0.49546931530716076,
 0.16780940509527678,
 2.626235810074259e-06,
 2.036531911579023,
 0.4645013981532215,
 5.5764228815324e-06,
 1340.5273727678225,
 2.356485418959427,
 0.005123293286719018)

### Exponential, $H_{\tau=0.4}$, $\sigma_E=2\times10^{-3}$, $order=3$

In [ ]:
beam.nCore_y=4
beam.nCore_z=2

In [ ]:
prepare_Exponential(0.49546931530716076,0.16780940509527678,emitGeomRMS,6.0,
                    2.036531911579023,0.4645013981532215,emitGeomRMS,6.0,
                    2.0e-3,0.4,0.4):

In [ ]:
run_Exponential(0,1,0.4,0.4,3)
run_Exponential(1,2,0.4,0.4,3)
run_Exponential(2,3,0.4,0.4,3)

In [ ]:
data = get_Exponential_data([0,1,2,3],0.4,0.4)